In [6]:
from autocorrect import Speller
import re

class TextProcessor:

    def __init__(self, lang: str = 'en'):
        self.autocorrect = Speller(lang=lang)


    def to_lowercase(self, text: str) -> str:
        return text.lower()
    
    def remove_punctuation(self, text: str) -> str:
        return re.sub(r'[^\w\s]', '', text)
    
    def correct_spelling(self, text: str) -> str:
        return self.autocorrect(text)

    def process(self, text: str) -> str:
        text = self.to_lowercase(text)
        text = self.remove_punctuation(text)
        text = self.correct_spelling(text)
        return text
    
    __call__ = process

class FrequencyAnalyzer:

    def __init__(self, corpus: str):
        self.corpus = corpus

    def split_into_words(self, text: str) -> list:
        return text.split()
    
    def word_frequencies(self, text: str) -> dict[str, int]:
        words = self.split_into_words(text)
        freq = {}
        for word in words:
            freq[word] = freq.get(word, 0) + 1
        return freq


In [7]:
Text = "Python is great! Python is easy to learn. Isn't Python amazing?"

processor = TextProcessor()
analyzer = FrequencyAnalyzer(corpus=Text)

processed_text = processor(Text)
processed_text

'python is great python is easy to learn isnt python amazing'

In [8]:
frequencies = analyzer.word_frequencies(processed_text)
frequencies

{'python': 3,
 'is': 2,
 'great': 1,
 'easy': 1,
 'to': 1,
 'learn': 1,
 'isnt': 1,
 'amazing': 1}